In [3]:
from transformers import AutoModelForSeq2SeqLM, AutoModelForCausalLM
from peft import PeftModel, PeftConfig
# base_model_name_or_path = '01ai/Yi-6B-Chat'

peft_model_id = "./output/Yi-6B/checkpoint-600"  # 这里我训练出效果最好的一版是 checkpoint-600，所以调用了这个，大家可以根据自己情况选择
config = PeftConfig.from_pretrained(peft_model_id)
from peft import LoraConfig, TaskType, get_peft_model
config = LoraConfig(
    task_type=TaskType.CAUSAL_LM, 
    target_modules=["q_attn", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    inference_mode=False, # 训练模式
    r=8, # Lora 秩
    lora_alpha=32, # Lora alaph，具体作用参见 Lora 原理
    lora_dropout=0.1# Dropout 比例
)

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer

mode_name_or_path = '/root/autodl-tmp/01ai/Yi-6B-Chat'

tokenizer = AutoTokenizer.from_pretrained(mode_name_or_path,
                                          use_fast=False, 
                                          trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(mode_name_or_path)
model = PeftModel.from_pretrained(model, peft_model_id)

Loading checkpoint shards: 100%|██████████| 3/3 [00:01<00:00,  1.53it/s]


In [5]:
model.eval()
input = tokenizer("<|im_start|>system\n现在你要扮演皇帝身边的女人--甄嬛.<|im_end|>\n<|im_start|>user\n{}<|im_end|>\n".format("你是谁？", "").strip() + "\nassistant\n ", return_tensors="pt").to(model.device)

max_length = 512

outputs = model.generate(
    **input,
    max_length=max_length,
    eos_token_id=7,
    do_sample=True,
    repetition_penalty=1.3,
    no_repeat_ngram_size=5,
    temperature=0.1,
    top_k=40,
    top_p=0.8,
)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

RuntimeError: probability tensor contains either `inf`, `nan` or element < 0